In [65]:
"""Profiling code"""
import cProfile, pstats, StringIO


def profile_callable(call, *args, **kwargs):
    """Profiles given callable with given arguments
    
    Result can be analyzed in PyCharm using Tools->Open CProfile snapshot
    
    Or with snakeviz: 
    pip install snakeviz
    snakeviz <.profile file>
    """
    pr = cProfile.Profile()
    pr.enable()

    ret = call(*args, **kwargs)

    pr.disable()
    s = StringIO.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.dump_stats("{}.profile".format(call.__name__))
    print("Finished profiling {}".format(call.__name__))
    
    return ret


def profileit(func):
    """Decorator that profiles decorated code"""
    def wrapper(*args, **kwargs):
        datafn = func.__name__ + ".profile" # Name the data file sensibly
        prof = cProfile.Profile()
        retval = prof.runcall(func, *args, **kwargs)
        prof.dump_stats(datafn)
        return retval

    return wrapper

In [66]:
"""Demo code"""
from pprint import pprint 

import yaml
from yaml import Loader
#from yaml import CLoader

def slow_function_loader():
    with open(r"C:\Users\tweining\workspace\automation\chippackages\Denali_A0\data\register_table.yaml", 'r') as stream:
        y = yaml.load(stream, Loader=Loader)
    pprint(y["bank0"][0])

def slow_function_cloader():
    with open(r"C:\Users\tweining\workspace\automation\chippackages\Denali_A0\data\register_table.yaml", 'r') as stream:
        y = yaml.load(stream, Loader=CLoader)
    pprint(y["bank0"][0])
    
def slow_function():
    with open(r"C:\Users\tweining\workspace\automation\chippackages\Denali_A0\data\register_table.yaml", 'r') as stream:
        y = yaml.load(stream)
    pprint(y["bank0"][0])

@profileit
def decorated_slow_function():
    with open(r"C:\Users\tweining\workspace\automation\chippackages\Denali_A0\data\register_table.yaml", 'r') as stream:
        y = yaml.load(stream)
    pprint(y["bank0"][0])
    

In [67]:
#%time profile_callable(slow_function_loader)

In [68]:
%time profile_callable(slow_function)

{'POR': 0,
 'access': '',
 'address': 4352,
 'bits': [{'access': 'R',
           'customer': 1,
           'desc': 'Read back of mask revision code (MRC) code 0 for TC release',
           'name': 'MRC',
           'reset': 'D',
           'start': 0,
           'stop': 7,
           'test': 0,
           'trim': 0}],
 'block': 'sysctl',
 'customer': 1,
 'desc': '',
 'name': 'SYSCTL_MASK_REV_CODE',
 'otp': 0,
 'otp_addr': ''}
Finished profiling slow_function
Wall time: 27 s


In [69]:
%time decorated_slow_function()

{'POR': 0,
 'access': '',
 'address': 4352,
 'bits': [{'access': 'R',
           'customer': 1,
           'desc': 'Read back of mask revision code (MRC) code 0 for TC release',
           'name': 'MRC',
           'reset': 'D',
           'start': 0,
           'stop': 7,
           'test': 0,
           'trim': 0}],
 'block': 'sysctl',
 'customer': 1,
 'desc': '',
 'name': 'SYSCTL_MASK_REV_CODE',
 'otp': 0,
 'otp_addr': ''}
Wall time: 26.3 s
